In [2]:
# !pip install gigachain faiss-cpu sentence-transformers sentencepiece rank_bm25 datasets --quiet

In [4]:
import datasets
from langchain.docstore.base import Document
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever, BM25Retriever

/Users/19814197/Documents/gigachain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Данные

In [5]:
ds = datasets.load_dataset("sberquad")
ds["train"][0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [6]:
validation_ds = ds["validation"]
documents = [
    Document(page_content=context)
    for context in set(validation_ds["context"])
]

## Ретривал

### Эмбеддинг модель

In [ ]:
from typing import List, Coroutine, Any


class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(self, texts: List[str]) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)

In [7]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")

In [8]:
faiss_db = FAISS.from_documents(documents, embedding=embedding)

In [9]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 5})

In [10]:
validation_ds = validation_ds.map(
    lambda x: {
        "embedding_retrieved": [
            passage.page_content
            for passage in embedding_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [10:12<00:00,  8.22 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved'],
    num_rows: 5036
})

In [11]:
def acc_top(dataset: datasets.Dataset, right_column: str, answer_column: str) -> float:
    temp_dataset = dataset.map(
        lambda x: {
            "is_right_retrieved": x[right_column] in x[answer_column]
        }
    )
    return sum(temp_dataset["is_right_retrieved"]) / len(temp_dataset)

In [12]:
acc_top(validation_ds, "context", "embedding_retrieved")

Map: 100%|██████████| 5036/5036 [00:02<00:00, 2309.51 examples/s]


0.9116362192216044

### BM25

In [13]:
import string


def tokenize(s: str) -> list[str]:
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")

In [14]:
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=5,
)

In [15]:
validation_ds = validation_ds.map(
    lambda x: {
        "bm25_retrieved": [
            passage.page_content
            for passage in bm25_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [17:47<00:00,  4.72 examples/s]  


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved', 'bm25_retrieved'],
    num_rows: 5036
})

In [16]:
acc_top(validation_ds, "context", "bm25_retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 3002.73 examples/s]


0.9197776012708498

### Ансамбль

In [17]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 2})
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=3,
)

In [18]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)

In [ ]:
validation_ds = validation_ds.map(
    lambda x: {
        "retrieved": [
            passage.page_content
            for passage in ensemble_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

In [ ]:
acc_top(validation_ds, "context", "retrieved")

## E2E решение

In [19]:
from langchain.chains import RetrievalQA
from langchain.llms.gigachat import GigaChat

In [20]:
llm = GigaChat(profanity=False, credentials=...)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
)

In [ ]:
qa.invoke({"query": "Что такое вода?"})